<a href="https://colab.research.google.com/github/G544/galinanana_ml/blob/main/kaggle/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"galinanana","key":"8147c232cb591bab100fb9318cb5ce2d"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 18.1MB/s]


In [4]:
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from copy import deepcopy
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [5]:
filezip = '/content/titanic.zip'

z = ZipFile(filezip)
print(z.infolist())
with ZipFile(filezip, 'r') as zip_file:
    zip_file.extract('train.csv', '.')
    zip_file.extract('test.csv', '.')
    zip_file.extract('gender_submission.csv', '.')

[<ZipInfo filename='gender_submission.csv' compress_type=deflate file_size=3258 compress_size=872>, <ZipInfo filename='test.csv' compress_type=deflate file_size=28629 compress_size=11171>, <ZipInfo filename='train.csv' compress_type=deflate file_size=61194 compress_size=22388>]


In [29]:
data = pd.read_csv('/content/train.csv')
data.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
323,324,1,2,"Caldwell, Mrs. Albert Francis (Sylvia Mae Harb...",female,22.0,1,1,248738,29.000,NaN,S
168,169,0,1,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.925,NaN,S
865,866,1,2,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,236852,13.000,NaN,S


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [31]:
data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [32]:
target = data['Survived']
data.drop('Survived', axis=1, inplace = True)

In [43]:
def preproc(data):
  data['Embarked'].fillna('Nan', inplace=True)
  data['Age'].fillna(24.0, inplace=True)
  data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True, axis=1)
  df = pd.concat([data,pd.get_dummies(data[['Sex', 'Embarked']])], axis=1)
  df.drop(['Sex', 'Embarked'], axis=1, inplace=True)
  if 'Embarked_Nan' in list(df.columns):
    df.drop('Embarke_Nan', axis=1, inplace=True)
  df.fillna(0, inplace=True)
  sc = StandardScaler()
  df = sc.fit_transform(df)
  df = pd.DataFrame(columns = sc.get_feature_names_out(),data = df)
  return df

In [35]:
df = preproc(data)
X_train, X_test, y_train,  y_test = train_test_split(df, target, test_size=0.3)

In [36]:
logreg = LogisticRegression()
ridge = RidgeClassifier()
nb = GaussianNB()
rf = RandomForestClassifier()
logreg.fit(X_train, y_train)
ridge.fit(X_train, y_train)
nb.fit(X_train, y_train)
rf.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
y_pred_rd = ridge.predict(X_test)
y_pred_nb = nb.predict(X_test)
y_pred_rf = rf.predict(X_test)

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
for y, name in zip((y_pred_lr, y_pred_rd, y_pred_nb, y_pred_rf), ('lr', 'ridge', 'nb','rf')):
  pred_ac = round(accuracy_score(y, y_test),3)
  pred_pr = round(precision_score(y, y_test),3)
  pred_recall = round(recall_score(y, y_test),3)
  print(f'Score for {name}  ')
  print('')
  print(f'Accuracy для {name} {pred_ac}  ')
  print(f'Precision для {name} {pred_pr}  ')
  print(f'Recall для {name} {pred_recall}  ')
  print('')

Score for lr  

Accuracy для lr 0.799  
Precision для lr 0.717  
Recall для lr 0.732  

Score for ridge  

Accuracy для ridge 0.799  
Precision для ridge 0.677  
Recall для ridge 0.753  

Score for nb  

Accuracy для nb 0.81  
Precision для nb 0.697  
Recall для nb 0.767  

Score for rf  

Accuracy для rf 0.825  
Precision для rf 0.768  
Recall для rf 0.76  



In [44]:
test = pd.read_csv('/content/test.csv')
id = test['PassengerId']
X_submiss = preproc(test)
diff = list(set(X_train.columns).difference(set(X_submiss.columns)))
dict_idx = ()
for itm in diff:
  idx = list(X_train.columns).index(itm)
  X_submiss.insert(idx, itm, 0)
submission_lr = logreg.predict(X_submiss)
submission_rd = ridge.predict(X_submiss)
submission_nb = nb.predict(X_submiss)
submission_rf = rf.predict(X_submiss)

In [45]:
d = {'PassengerId':id, 'Survived':submission_lr}
log_reg_df = pd.DataFrame(d)
d = {'PassengerId':id, 'Survived':submission_rd}
ridge_df = pd.DataFrame(d)
d = {'PassengerId':id, 'Survived':submission_nb}
nb_df = pd.DataFrame(d)
d = {'PassengerId':id, 'Survived':submission_rf}
rf_df = pd.DataFrame(d)

In [46]:
log_reg_df.to_csv('lr_submiss.csv', index=False)
ridge_df.to_csv('rd_submiss.csv', index=False)
nb_df.to_csv('nb_submiss.csv', index=False)
rf_df.to_csv('rf_submiss.csv', index=False)

In [47]:
!kaggle competitions submit -c titanic -f lr_submiss.csv -m "Age, Embarked"
!kaggle competitions submit -c titanic -f rd_submiss.csv -m "Age, Embarked"
!kaggle competitions submit -c titanic -f nb_submiss.csv -m "Age, Embarked"
!kaggle competitions submit -c titanic -f rf_submiss.csv -m "Age, Embarked"

100% 2.77k/2.77k [00:01<00:00, 1.70kB/s]
100% 2.77k/2.77k [00:01<00:00, 2.05kB/s]
100% 2.77k/2.77k [00:01<00:00, 1.88kB/s]
100% 2.77k/2.77k [00:01<00:00, 2.22kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

Результаты бейзлайна  
Score for {name} lr  
Accuracy для {name} {pred_ac} lr 0.686567164179104  
Accuracy для {name} {pred_pr
 lr 0.6857142857142857  
Accuracy для {name} {pred_recall} lr 0.7128712871287128  

Score for {name} ridge

Accuracy для {name} {pred_ac} ridge 0.7649253731343284
Accuracy для {name} {pred_pr} ridge 0.6857142857142857
Accuracy для {name} {pred_recall} ridge 0.7058823529411765

Score for {name} nb

Accuracy для {name} {pred_ac} nb 0.7761194029850746
Accuracy для {name} {pred_pr} nb 0.7333333333333333
Accuracy для {name} {pred_recall} nb 0.7064220183486238


С использованием стандартскейлер

Score for lr  

Accuracy для lr 0.836  
Precision для lr 0.777  
Recall для lr 0.792  

Score for ridge  

Accuracy для ridge 0.817  
Precision для ridge 0.738  
Recall для ridge 0.776  

Score for nb  

Accuracy для nb 0.642  
Precision для nb 0.107  
Recall для nb 0.733  

Score for rf  

Accuracy для rf 0.81  
Precision для rf 0.738  
Recall для rf 0.76  